# Model 3
## Dataset: Car images - Ford v. Audi

## Programmer: Giovanni Vecchione
## Date: 3/21/24
## Subject: Machine Learning 2 - Project 3
Goal is to create a CNN that can identify the car brand
Adjustment: Lowered brands to just two to improve training time

## Important: Ensure your 'train' and 'test' folders have the following structure:

train/
    car_type_1/
        image1.jpg
        image2.jpg
        ...
    car_type_2/
        ...
    ...
test/
    car_type_1/
        ...
    car_type_2/
        ...
    ...

In [1]:
import tensorflow as tf
import numpy as np
from functools import partial
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator

seed = 42
random.seed(seed)


In [2]:
#Checks if GPU is being used
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use the GPU
    print("Using GPU:", torch.cuda.get_device_name(0)) 
else:
    device = torch.device("cpu")  # Fallback to CPU
    print("GPU not available, using CPU.")

#Using GPU: NVIDIA GeForce GTX 1660 SUPER - Successful
#NOTE: This took some time to set up by installing and pathing the cuda toolkit v.12.4 and the right supplemental packages. This drastically improved
#training time

Using GPU: NVIDIA GeForce GTX 1660 SUPER


In [3]:
#This checks data set location and verifies if it's identified
import os
for file in os.listdir(r"D:\GioDude\Documents\ACC\Spring 2024\Machine Learning II\Datasets\vehicle_classification_kaggle" ):
    print(file)

test
train
val


# Preprocessing the Data 3/21:
Images need to have the csv so that it has the model with labels.

slice the file name dependent on the class names needed for regular files.

Changed dataset to a much more manageable source. However there is much less images so this could impede some training performance.

Orginally this was the code to reshuffle the data since it was data that was purely uncategorized:
#shuffle the dataset with a random seed
dataset_shuffled = dataset_images.shuffle(10000, seed=42)

Take only 5000 images from the shuffled dataset
dataset_subset = dataset_shuffled.take(5000)

Split the dataset into training, validation, and test datasets
train_size = 3000
val_size = 1000
test_size = 1000

train_ds = dataset_subset.take(train_size)
val_ds = dataset_subset.skip(train_size).take(val_size)
test_ds = dataset_subset.skip(train_size + val_size).take(test_size)

### NOTE: This proved unsuccessful used the code below instead.

In [4]:
#Import Data by Path - Change this if location is different
dataset_path_train = r"D:\GioDude\Documents\ACC\Spring 2024\Machine Learning II\Datasets\ford_or_audi_images\train"
dataset_path_val = r"D:\GioDude\Documents\ACC\Spring 2024\Machine Learning II\Datasets\ford_or_audi_images\val"
dataset_path_test = r"D:\GioDude\Documents\ACC\Spring 2024\Machine Learning II\Datasets\ford_or_audi_images\test"

#Ended up having to reformat the data, reduced the classes and cleaned the data aswell. This improved training accuracy from 10% to 60% percent alone.
#NOTE: Having trouble with validation data.

In [5]:
# Set image size for resizing - standard 244,244
IMG_SIZE = (224, 224)

# Create generators for training and validation sets
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)
#data agumentations to add more training data to the process

val_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
BATCH_SIZE = 10 #NOTE: batch size incompatable / batch size needs to split evenly with the size of the datasets otherwise you will have incorrect
#input sizes
#NOTE: The batch size was resolved by adding it to the dimensions of the images below

train_ds = train_datagen.flow_from_directory(
    dataset_path_train,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,  # Choose any batch size now
    shuffle=True, 
    seed = 42,
    class_mode='categorical',  # Assuming multi-class
) 

val_ds = val_datagen.flow_from_directory(
    dataset_path_val,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed = 42, 
    class_mode='categorical' 
) 

# Resize the images and set batch size #NOTE: This is not used in the code but works for other cases.
#train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, IMG_SIZE), y)).batch(BATCH_SIZE)
#val_ds = val_ds.map(lambda x, y: (tf.image.resize(x, IMG_SIZE), y)).batch(BATCH_SIZE)

Found 5400 images belonging to 2 classes.
Found 987 images belonging to 2 classes.


In [7]:
#check image shape
image, label = next(iter(train_ds))
print(image.shape, label.shape)

(10, 224, 224, 3) (10, 2)


In [8]:
#check image labels
class_names = train_ds.class_indices.keys() 
class_names2 = val_ds.class_indices.keys() 
print(class_names) 
print(class_names2)

dict_keys(['Audi', 'Ford'])
dict_keys(['Audi', 'Ford'])


In [9]:
model3 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding ='same', activation='relu', input_shape= (224, 224, 3), kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer="he_normal"),
    tf.keras.layers.Dense(2, activation='softmax') 
])

model3.compile(loss='categorical_crossentropy', #good for multiclass
              optimizer='adam',
              metrics=['accuracy'])


model3.summary()

c:\Users\GioDude\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 110, 110, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 55, 55, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 53, 53, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    22,151,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,277,954 (84.98 MB)

 Trainable params: 22,277,890 (84.98 MB)

 Non-trainable params: 64 (256.00 B)

In [10]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history3 = model3.fit(train_ds, validation_data=val_ds, epochs=20, callbacks = [early_stopping])

Epoch 1/20


c:\Users\GioDude\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


540/540 ━━━━━━━━━━━━━━━━━━━━ 182s 333ms/step - accuracy: 0.6090 - loss: 1.2639 - val_accuracy: 0.5876 - val_loss: 0.7702
Epoch 2/20
115/540 ━━━━━━━━━━━━━━━━━━━━ 2:21 332ms/step - accuracy: 0.6102 - loss: 0.7612

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

pd.DataFrame(history3.history).plot(
    figsize=(8, 5), xlim=[0, 29], ylim=[0, 1], grid=True, xlabel="Epoch",
    style=["r--", "r--.", "b-", "b-*"])
plt.show()

In [ ]:
# Evaluate the model on the validation set
val_loss, val_acc = model3.evaluate(X_valid, y_valid)
print("Validation loss: {:.3f}, Validation accuracy: {:.3f}".format(val_loss, val_acc))

# Model 3 Testing:
## NOTE: SIGNIFICANT TIME was spent adjusting the dataset to improve its overall readability. Was problem solving shape of the model as well. Also, installing the proper packages needed to run the CNN on my GPU took some time aswell. Kaggle unfortuantely has alot of poor datasets to implement this project, took some time to find a good dataset, clean it, and organize it. This is the best I could do with the time available. Testing shown as follows.

## Unalbe to solve the validation accuracy and validation loss issue with my program. It seems to not change however it is processing it.

Test 1: 
Overfitting, changing the dense layers to decrease complexity.

Results: This stopped the overfitting, however the model is performing poorly in both valid and training. However this is a step in the right direction from overfitting.

Test 2:
Adding layers to see if this helps with complexity. Adjusted filters also from 32,64 to 32,64,64,128,128

...
tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
...

Test 3: Overal this model so far is performing poorly and sees no signs of improving. Lets try to re-do these layers since we are dealing with a smaller dataset.

No improvement

Test 4:

Cleaned the data removed classes to simplify classification, and added more images to the classes. Cleaned up the test and validation data also. This substantially improved the performance. It seems the data set even though it was organized well suffered from too many classes with too little data.

However the performance is still very poor.

Test 5:

It seems that the validation loss is high and the validation accuracy is not budging. Further more the accuracy and loss has stagnated.
Possible need to change learning rate and prevent overfitting. Also need to rebalance the test, validation, and test datasets.

Epoch 2/20
700/700 ━━━━━━━━━━━━━━━━━━━━ 410s 585ms/step - accuracy: 0.6063 - loss: 0.6727 - val_accuracy: 0.4650 - val_loss: 0.7235
Epoch 3/20
700/700 ━━━━━━━━━━━━━━━━━━━━ 408s 582ms/step - accuracy: 0.6178 - loss: 0.6662 - val_accuracy: 0.4650 - val_loss: 0.7114
Epoch 4/20
700/700 ━━━━━━━━━━━━━━━━━━━━ 408s 582ms/step - accuracy: 0.6161 - loss: 0.6673 - val_accuracy: 0.4650 - val_loss: 0.7160
Epoch 5/20
700/700 ━━━━━━━━━━━━━━━━━━━━ 408s 582ms/step - accuracy: 0.6196 - loss: 0.6666 - val_accuracy: 0.4650 - val_loss: 0.7220
Epoch 6/20
700/700 ━━━━━━━━━━━━━━━━━━━━ 408s 583ms/step - accuracy: 0.6110 - loss: 0.6709 - val_accuracy: 0.4650 - val_loss: 0.7384
Epoch 7/20
700/700 ━━━━━━━━━━━━━━━━━━━━ 409s 583ms/step - accuracy: 0.6143 - loss: 0.6678 - val_accuracy: 0.4650 - val_loss: 0.7396


Test 6:

Model is performing poorly on the validation data possibly suggesting overfitting. Attempting again with another dense layer (128) and adding some L2 regularization. 

Test 7:
validation loss / accuracy not changing. attempting clean up in validation preprocessing

Test 8:

Validation loss /accuracy not changing.

Test 9:

Validation loss /accuracy not changing.

Test 10:

Validation loss /accuracy not changing. Calling it a day.
